<a href="https://colab.research.google.com/github/SwethaNandyala/DeepLearning/blob/main/Data_Augumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install kaggle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir ~/.kaggle

!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:29<00:00, 37.1MB/s]
100% 1.06G/1.06G [00:29<00:00, 38.6MB/s]


In [6]:
from zipfile import ZipFile
with ZipFile("/content/dogs-vs-cats.zip",'r')as zipobj:
  zipobj.extractall('/content/dogs-vs-cats')

In [7]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from keras.optimizers import Adam

In [8]:
size = 256
image_shape = (size,size,3)
BATCH_SIZE = 32

In [9]:
cnn = Sequential()

cnn.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',kernel_initializer='he_uniform',padding='same',input_shape=image_shape))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu',kernel_initializer='he_uniform',padding='same',input_shape=image_shape))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(filters=16,kernel_size=(3,3),activation='relu',kernel_initializer='he_uniform',padding='same',input_shape=image_shape))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Flatten())

cnn.add(Dense(units=64,activation='relu',kernel_initializer='he_uniform'))
cnn.add(Dense(units=32,activation='relu',kernel_initializer='he_uniform'))
cnn.add(Dense(units=1,activation='sigmoid',kernel_initializer='glorot_uniform'))

cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 16)       0

In [10]:
opt = Adam(learning_rate=0.001)
cnn.compile(optimizer=opt,metrics=['accuracy'],loss='binary_crossentropy')

In [11]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [12]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                    rotation_range=45,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='nearest',
                                    validation_split=0.20
                                   )

test_datagen = ImageDataGenerator(rescale=1.0/255)

In [13]:
train_data = train_datagen.flow_from_directory(directory= '/content/dogs-vs-cats/train',
                                                target_size=(256,256),
                                                color_mode='rgb',
                                                class_mode='binary',
                                                batch_size=BATCH_SIZE,
                                                subset = "training"
                                              )
valid_data = train_datagen.flow_from_directory(directory='/content/dogs-vs-cats/train',
                                                target_size=(256,256),
                                                color_mode='rgb',
                                                class_mode='binary',
                                                batch_size=BATCH_SIZE,
                                                subset="validation"
                                              )

test_data = train_datagen.flow_from_directory(directory='/content/dogs-vs-cats/test',
                                                target_size=(256,256),
                                                color_mode='rgb',
                                                class_mode='binary',
                                                batch_size=BATCH_SIZE,

                                              )

Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [14]:
cnn.fit_generator(train_data, steps_per_epoch=16000//32,
    epochs=50,
    verbose=1,
    validation_data=valid_data,
    validation_steps=4000//32)

<ipython-input-14-892923ba86ae>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn.fit_generator(train_data, steps_per_epoch=16000//32,


Epoch 1/50
500/500 [==============================] - 395s 767ms/step - loss: 0.6783 - accuracy: 0.5683 - val_loss: 0.6484 - val_accuracy: 0.6255
Epoch 2/50
500/500 [==============================] - 384s 768ms/step - loss: 0.6261 - accuracy: 0.6442 - val_loss: 0.5860 - val_accuracy: 0.6830
Epoch 3/50
500/500 [==============================] - 371s 742ms/step - loss: 0.5854 - accuracy: 0.6898 - val_loss: 0.5517 - val_accuracy: 0.7155
Epoch 4/50
500/500 [==============================] - 368s 737ms/step - loss: 0.5554 - accuracy: 0.7153 - val_loss: 0.5477 - val_accuracy: 0.7200
Epoch 5/50
500/500 [==============================] - 379s 759ms/step - loss: 0.5434 - accuracy: 0.7208 - val_loss: 0.5374 - val_accuracy: 0.7270
Epoch 6/50
500/500 [==============================] - 368s 736ms/step - loss: 0.5356 - accuracy: 0.7311 - val_loss: 0.5173 - val_accuracy: 0.7442
Epoch 7/50
500/500 [==============================] - 369s 738ms/step - loss: 0.5280 - accuracy: 0.7357 - val_loss: 0.5208 -